### RNN模型纵览

<br>

### 训练中的输出

<br>

### 损失函数

<br>

### 梯度裁剪

<br>

### 潜变量初始化

<br>

### 使用模型进行预测

<br>

### 代码实现

In [22]:
import torch
from torch.nn import functional


class RNNModel:
    # hiden_size 是潜变量的大小
    def __init__(self, time_step, vocab_size, hiden_size):
        self.time_step = time_step
        self.vocab_size = vocab_size
        self.hiden_size = hiden_size

        # 用于更新潜变量的模型参数
        self.w_hh = torch.randn(hiden_size, hiden_size) * 0.01
        self.w_xh = torch.randn(vocab_size, hiden_size) * 0.01
        self.b_h = torch.zeros(hiden_size)

        # 通过潜变量获取输出的模型参数
        self.w_y = torch.randn(hiden_size, vocab_size) * 0.01
        self.b_y = torch.randn(vocab_size) * 0.01

        self.w_hh.requires_grad_(True)
        self.w_xh.requires_grad_(True)
        self.b_h.requires_grad_(True)
        self.w_y.requires_grad_(True)
        self.b_y.requires_grad_(True)


    # 在没有任何历史信息的情况下，初始化潜变量（全部初始化为 0）
    def init_state(self, batch_size):
        return torch.zeros(batch_size, self.hiden_size)

    # 对一个批次的样本进行正向传播
    def __call__(self, X):
        return self.forward(X)
    
    # X 的形状为 (time_step, batch_size, vocab_size)
    # 这样可以同时更新不同样本的潜变量
    def forward(self, X):
        h = self.init_state(X.shape[1])    # 初始化潜变量
        Y = []     # 用于保存每一个时间步长上的预测值
        for x in X:
            h = h @ self.w_hh + x @ self.w_xh + self.b_h
            y = h @ self.w_y + self.b_y
            Y.append(y)
        return torch.tensor(Y), (h,)
    
    # 梯度裁剪，防止梯度爆炸，只有整体梯度的 L2 范数大于 theta 时，才需要进行裁剪
    def grad_clipping(self, theta):
        norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in self.get_parameters()))
        if norm > theta:
            for param in self.get_parameters():
                param.grad[:] *= theta / norm
    
    def get_parameters(self):
        return [self.w_hh, self.w_xh, self.b_h, self.w_y, self.b_y]
    
    # 预测函数，根据 num_prefix 个 token 的输入，预测后续 num_predic 个 token 的输出
    # prefix 的形状为 (len, vocab_size)
    def prefict(self, prefix, num_predic):
        # 潜变量的预热
        h = self.init_state(1)
        for x in prefix:
            h = h @ self.w_hh + x.reshape(1, -1) @ self.w_xh + self.b_h
        
        # 进行后续 num_predic 个词的预测
        Y = []
        for _ in range(num_predic):
            y = (h @ self.w_y + self.b_y).flatten().argmax()
            h = h @ self.w_hh + functional.one_hot(y, self.vocab_size).reshape(1, -1) @ self.w_xh + self.b_h
            Y.append(y)
        return torch.tensor(Y)

In [23]:
# 损失函数的计算
# y_hat 和 y 的形状都是 (time_step, batch_size, vocab_size)
# 每个样本在不同的时间步长上都有输出（vocab_size 个得分）
def loss_func(y_hat, y):
    # 将数据形状变为(time_step * batch_size, vocab_size)
    # 方便计算 softmax 和 crossEntropyLoss
    y1 = torch.cat(y_hat, dim=0)
    y2 = torch.cat(y, dim=0)
    return torch.nn.CrossEntropyLoss(y1, y2)

In [24]:
# 进行一次参数更新（训练一个 batch）
def train_batch(rnn_model: RNNModel, X:torch.Tensor, Y:torch.Tensor, optimizer:torch.optim.Optimizer):
    optimizer.zero_grad()
    y, _ = rnn_model.forward(X)
    loss = loss_func(y, Y)
    loss.backword()
    rnn_model.grad_clipping(1)
    optimizer.step()

然后使用不同的batch循环调用上面的训练即可